In [8]:
import os
import numpy
from scipy import integrate, linalg
import math
import matplotlib.pyplot as pyplot

class Panel:
    def __init__(self, xa, ya, xb, yb):
        self.xa, self.ya = xa, ya  
        self.xb, self.yb = xb, yb  
        self.xc, self.yc = (xa + xb) / 2, (ya + yb) / 2 
        self.length = numpy.sqrt((xb - xa)**2 + (yb - ya)**2)  
        if xb - xa <= 0.0:
            self.beta = numpy.arccos((yb - ya) / self.length)
        elif xb - xa > 0.0:
            self.beta = numpy.pi + numpy.arccos(-(yb - ya) / self.length)
        self.sigma = 0.0  
        self.vt = 0.0  
        self.cp = 0.0  

def define_panels(x, y):
    x, y = numpy.append(x, x[0]), numpy.append(y, y[0])
    panels = numpy.empty(len(x)-1 , dtype=object)
    for i in range(len(x)-1):
        panels[i] = Panel(x[i], y[i], x[i + 1], y[i + 1])
    return panels

class Freestream:
    def __init__(self, u_inf=1.0, alpha=0.0):
        self.u_inf = u_inf
        self.alpha = numpy.radians(alpha)

def integral(x, y, panel, dxdk, dydk):
    def integrand(s):
        return (((x - (panel.xa - numpy.sin(panel.beta) * s)) * dxdk +
                 (y - (panel.ya + numpy.cos(panel.beta) * s)) * dydk) /
                ((x - (panel.xa - numpy.sin(panel.beta) * s))**2 +
                 (y - (panel.ya + numpy.cos(panel.beta) * s))**2) )
    tmp=integrate.quad(integrand, 0.0, panel.length)[0]
    return tmp

def source_contribution_normal(panels):
    A = numpy.empty((panels.size, panels.size), dtype=float)
    numpy.fill_diagonal(A, 0.5)
    for i, panel_i in enumerate(panels):
        for j, panel_j in enumerate(panels):
            if i != j:
                A[i, j] = 0.5 / numpy.pi * integral(panel_i.xc, panel_i.yc, 
                                                    panel_j,
                                                    numpy.cos(panel_i.beta),
                                                    numpy.sin(panel_i.beta))
    return A

def vortex_contribution_normal(panels):
    A = numpy.empty((panels.size, panels.size), dtype=float)
    numpy.fill_diagonal(A, 0.0)
    for i, panel_i in enumerate(panels):
        for j, panel_j in enumerate(panels):
            if i != j:
                A[i, j] = -0.5 / numpy.pi * integral(panel_i.xc, panel_i.yc, 
                                                     panel_j,
                                                     numpy.sin(panel_i.beta),
                                                     -numpy.cos(panel_i.beta))
    return A

def kutta_condition(A_source, B_vortex):
    b = numpy.empty(A_source.shape[0] + 1, dtype=float)
    b[:-1] = B_vortex[0, :] + B_vortex[-1, :]
    b[-1] = - numpy.sum(A_source[0, :] + A_source[-1, :])
    return b

def build_singularity_matrix(A_source, B_vortex):
    A = numpy.empty((A_source.shape[0] + 1, A_source.shape[1] + 1), dtype=float)
    A[:-1, :-1] = A_source
    A[:-1, -1] = numpy.sum(B_vortex, axis=1)
    A[-1, :] = kutta_condition(A_source, B_vortex)
    return A

def build_freestream_rhs(panels, freestream):
    b = numpy.empty(panels.size + 1, dtype=float)
    for i, panel in enumerate(panels):
        b[i] = -freestream.u_inf * numpy.cos(freestream.alpha - panel.beta)
    b[-1] = -freestream.u_inf * (numpy.sin(freestream.alpha - panels[0].beta) +
                                 numpy.sin(freestream.alpha - panels[-1].beta) )
    return b

def compute_tangential_velocity(panels, freestream, gamma, A_source, B_vortex):
    A = numpy.empty((panels.size, panels.size + 1), dtype=float)
    A[:, :-1] = B_vortex
    A[:, -1] = -numpy.sum(A_source, axis=1)
    b = freestream.u_inf * numpy.sin([freestream.alpha - panel.beta 
                                      for panel in panels])
    strengths = numpy.append([panel.sigma for panel in panels], gamma)
    tangential_velocities = numpy.dot(A, strengths) + b
    for i, panel in enumerate(panels):
        panel.vt = tangential_velocities[i]


def compute_pressure_coefficient(panels, freestream):
    for panel in panels:
        panel.cp = 1.0 - (panel.vt / freestream.u_inf)**2


def CL(x,y,aoa):
    panels = define_panels(x, y)
    freestream = Freestream(u_inf=1.0, alpha=aoa)
    A_source = source_contribution_normal(panels)
    B_vortex = vortex_contribution_normal(panels)
    A = build_singularity_matrix(A_source, B_vortex)
    b = build_freestream_rhs(panels, freestream)
    strengths = numpy.linalg.solve(A, b)
    for i , panel in enumerate(panels):
        panel.sigma = strengths[i]
    gamma = strengths[-1]
    compute_tangential_velocity(panels, freestream, gamma, A_source, B_vortex)
    compute_pressure_coefficient(panels, freestream)
    c = abs(max(panel.xa for panel in panels) -
            min(panel.xa for panel in panels))
    return (gamma * sum(panel.length for panel in panels) /
            (0.5 * freestream.u_inf * c))

def bem(tol, c, twist, w,r,u,B,x,y):
    i=0
    sigma=B*c/(2*math.pi*r)
    lambda_r=w*r/u
    a=1/3
    a_prime=0
    err=tol+1
    while err>tol:
        i+=1
        phi=math.atan((1-a)/(lambda_r*(1+a_prime)))
        aoa=phi-twist
        cl=CL(x,y,aoa)
        tmp1=sigma/(4*math.sin(phi)**2)*(cl*math.cos(phi))
        a=tmp1/(1+tmp1)
        a_prime=(1-a)*sigma/(4*lambda_r*math.sin(phi)**2)*(cl*math.cos(phi))
        err=abs(math.tan(phi)-(1-a)/(lambda_r*(1+a_prime)))
        if i>100:
            print(1)
            return 0
    return a

def efficiency(xs, ys, twists,tol,chords,w,R,u,B):
    n=len(xs)
    res=0
    for i in range(n):
        tmp=bem(tol,chords[i],twists[i],w,R/n*(i+1),u,B,xs[i],ys[i])
        res+=(2*i+1)*4*tmp*(1-tmp)**2
    return res/n**2
